In [7]:
import pathway as pw
print(dir(pw.xpacks.llm.vector_store))


['Callable', 'Coroutine', 'DataIndex', 'Iterable', 'SlidesVectorStoreServer', 'TYPE_CHECKING', 'TypeAlias', 'VectorStoreClient', 'VectorStoreServer', '_SCORE', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_coerce_sync', '_knn_lsh', '_unwrap_udf', 'cast', 'coerce_async', 'default_usearch_knn_document_index', 'jmespath', 'json', 'logging', 'pathway', 'pw', 'requests', 'threading', 'warnings']


In [10]:
# pathway_pipeline.ipynb

%pip install pathway

import sys

from typing import List, Tuple
from transformers import pipeline

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Remove duplicates from sys.path
sys.path = list(set(sys.path))

# Add the absolute path to the src directory
sys.path.append('/Users/debdeepbanerjee/Desktop/Projects/pathway_ds_iitkgp/src')

# Verify that the backend module is in the src directory
import os
if not os.path.isdir('/Users/debdeepbanerjee/Desktop/Projects/pathway_ds_iitkgp/src/backend'):
    raise ModuleNotFoundError("The 'backend' module is not found in the specified path.")

# Verify that the path has been added only once
print("Updated sys.path:", sys.path)

# Print the Python executable being used
print("Python Executable:", sys.executable)



Updated sys.path: ['', '/Users/debdeepbanerjee/.pyenv/versions/3.10.0/lib/python3.10', '/var/folders/d0/dlqx3vfs4kq4dx9v77jybst40000gn/T/tmpncgu193w', '/Users/debdeepbanerjee/.pyenv/versions/3.10.0/lib/python3.10/lib-dynload', '/Users/debdeepbanerjee/Desktop/pathway_ds_iitkgp/myenv/lib/python3.10/site-packages', '/Users/debdeepbanerjee/Desktop/Projects/pathway_ds_iitkgp/src', '/Users/debdeepbanerjee/.pyenv/versions/3.10.0/lib/python310.zip', '/Users/debdeepbanerjee/Desktop/Projects/pathway_ds_iitkgp/src']
Python Executable: /Users/debdeepbanerjee/Desktop/pathway_ds_iitkgp/myenv/bin/python


In [15]:
# Import necessary modules
import pathway as pw
from backend.pathway_integration import PathwayIntegration
from pathway.xpacks.llm.vector_store import VectorStoreServer

# Ensure that 'pw.io' contains the 'gdrive' method
print("Available modules in pw.io:", dir(pw.io))


Available modules in pw.io: ['CsvParserSettings', 'OnChangeCallback', 'OnFinishCallback', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_subscribe', '_utils', 'airbyte', 'bigquery', 'csv', 'debezium', 'deltalake', 'elasticsearch', 'fs', 'gdrive', 'http', 'iceberg', 'jsonlines', 'kafka', 'logstash', 'minio', 'mongodb', 'nats', 'null', 'plaintext', 'postgres', 'pubsub', 'pyfilesystem', 'python', 'redpanda', 's3', 's3_csv', 'slack', 'sqlite', 'subscribe']


In [27]:
class PathwayIntegration:
    def __init__(self, embedder_model="allenai/scibert_scivocab_uncased"):
        # Initialize Google Drive access using the available 'read' method from 'pw.io.gdrive'
        self.google_drive_connector = pw.io.gdrive.read  # Using 'read' method directly from 'pw.io'
        
        # Set up embedding pipeline (using SciBERT or any other model)
        self.embedder = pipeline("feature-extraction", model=embedder_model, tokenizer=embedder_model)

        # Initialize VectorStoreServer
        # Provide a data source (e.g., from Google Drive or a folder)
        self.vector_store_server = VectorStoreServer(
            docs=self.stream_data(),  # Pass the docs stream here
            embedder=self._embedding_udf,
            parser=self._basic_parser,
        )

    def stream_data(self):
        """Stream papers data using Pathway connectors."""
        # Replace 'your_object_id' with the actual object ID (file/folder ID) you want to read
        object_id = "1Z8z4craj36ighb8hzUzeM76OOgpUdsKr"  # Replace with actual object ID from Google Drive
        credentials_file = "/Users/debdeepbanerjee/Downloads/credentials.json"  # Path to your credentials JSON file
        # Fetch the documents from Google Drive
        return self.google_drive_connector(object_id, service_user_credentials_file=credentials_file)

    def _embedding_udf(self, text: str) -> List[float]:
        """Embed text using the embedding pipeline."""
        embeddings = self.embedder(text)
        return embeddings[0][0]  # Flatten the output

    def _basic_parser(self, content: bytes) -> List[Tuple[str, dict]]:
        """Basic parser to handle file content."""
        text = content.decode("utf-8")
        return [(text, {})]

    def query_statistics(self):
        """Query document statistics."""
        query_table = pw.Table.from_dicts([{"id": 1}])  # Example query schema
        result_table = self.vector_store_server.statistics_query(query_table)
        return result_table

    def query_similar_embeddings(self, query_embedding: List[float]):
        """Query nearest neighbors for an embedding."""
        return self.vector_store_server._graph["knn_index"].query(query_embedding, top_k=3)

    def query_inputs(self, metadata_filter=None, filepath_globpattern=None):
        """Query inputs using metadata filtering."""
        query_table = pw.Table.from_dicts([
            {"id": 1, "metadata_filter": metadata_filter, "filepath_globpattern": filepath_globpattern}
        ])
        result_table = self.vector_store_server.inputs_query(query_table)
        return result_table

    def connect_papers(self, paper_a: str, paper_b: str):
        """Connect two papers based on some criteria."""
        # Implement the logic to connect papers
        connection_result = f"Connecting {paper_a} with {paper_b}."
        return connection_result

# Initialize PathwayIntegration
integration = PathwayIntegration()

# Use the connection to link two papers (replace "Paper A" and "Paper B" with actual paper names)
result = integration.connect_papers("Paper A", "Paper B")

# Print result of the connection
print("Connection Result:", result)

# Example of querying Google Drive data (ensure the path is valid)
try:
    drive_data = integration.google_drive_connector("your_object_id")  # Pass the object ID here
    print("Drive Data:", drive_data)
except Exception as e:
    print(f"Error accessing Google Drive: {e}")

# Check available methods in Pathway Integration and other debugging information
print("Available methods in PathwayIntegration:", dir(integration))

# Check and display query statistics (just an example)
query_stats = integration.query_statistics()
print("Query Statistics Result:", query_stats)

# Display sample query for similar embeddings (you may replace the query with actual query data)
sample_query_embedding = [0.1, 0.2, 0.3, 0.4, 0.5]  # Replace with actual embedding data
similar_embeddings = integration.query_similar_embeddings(sample_query_embedding)
print("Similar Embeddings Query Result:", similar_embeddings)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use mps:0


TypeError: VectorStoreServer.__init__() got an unexpected keyword argument 'docs'

In [12]:
# Print available methods and attributes in pw.io.gdrive
print(dir(pw.io.gdrive))


['Any', 'Callable', 'ConnectorSubject', 'DEFAULT_MIME_TYPE_MAPPING', 'FILE_FIELDS', 'GDriveFile', 'HttpError', 'MIME_TYPE_FOLDER', 'MediaIoBaseDownload', 'NewType', 'SCOPES', 'STATUS_DOWNLOADED', 'STATUS_SIZE_LIMIT_EXCEEDED', 'STATUS_SYMLINKS_NOT_SUPPORTED', 'ServiceCredentials', 'SessionType', '_GDriveClient', '_GDriveSubject', '_GDriveTree', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'add_path', 'add_seen_at', 'add_status', 'add_url', 'annotations', 'api', 'build', 'check_arg_types', 'dataclass', 'extend_metadata', 'fnmatch', 'io', 'json', 'logging', 'pw', 'read', 'time', 'warnings']


In [8]:
object_id = "1Z8z4craj36ighb8hzUzeM76OOgpUdsKr"

# Path to your Google Service Account credentials JSON file
credentials_path = "/path/to/your/service_account_credentials.json"

try:
    # Pass the credentials file to the read method
    drive_data = pw.io.gdrive.read(object_id, service_user_credentials_file=credentials_path)
    print("Drive Data:", drive_data)
except Exception as e:
    print(f"Error reading from Google Drive: {e}")


Drive Data: <pathway.Table schema={'data': typing.Any}>


In [6]:
# Inspect the GDriveFile class
print(dir(pw.io.gdrive.GDriveFile))  # Check methods and attributes of GDriveFile class


['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__or__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__supertype__', '__weakref__']
